# import libs

In [1]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/}"
)  # just to enable `dataset`

import tensorflow as tf
import optuna
import dataset
import numpy as np
import keras
import joblib
from datetime import datetime
from keras import layers

/home/webphy/Desktop/dnn_processor/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##  define constants and create dataset


In [2]:
NTRIALS = 800
DEFAULT_BATCH_SIZE = 64
EPOCHS = 100
DEFAULT_INPUT_SIZE = 128

train_images_rgx = "../../dataset/train_set_4x/*/*.jpeg"
test_images_rgx = "../../dataset/test_set_4x/*/*.jpeg"

layer_options = ["Conv2D", "Depthwise2D", "Pointwise2D"]
activation_options = ["linear", "relu"]
filter_size_options = [3, 4, 5]
num_layer_options = [2, 3]
dropout_range = [0.2, 0.6]
fmap_size_range = [512, 1024, 64]

train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, DEFAULT_BATCH_SIZE, DEFAULT_INPUT_SIZE
)

## define create model function


In [3]:
def clip_filter_size(fs, ih):
    return min(fs, ih)


def create_conv2d(trial, layer_id, max_stride, input_size):
    num_filters = trial.suggest_int(f"layer_{layer_id}_conv2d_filters", 4, 32, step=4)
    filter_size = trial.suggest_categorical(
        f"layer_{layer_id}_conv2d_fsize", filter_size_options
    )

    stride_size = trial.suggest_int(f"layer_{layer_id}_conv2d_ssize", 2, max_stride)

    return layers.Conv2D(
        num_filters,
        clip_filter_size(filter_size, input_size),
        stride_size,
    )


def create_dw(trial, layer_id, max_stride, input_size):
    filter_size = trial.suggest_categorical(
        f"layer_{layer_id}_dwconv2d_fsize", filter_size_options
    )
    stride_size = trial.suggest_int(f"layer_{layer_id}_dwconv2d_ssize", 2, max_stride)
    return layers.DepthwiseConv2D(
        clip_filter_size(filter_size, input_size), stride_size
    )


def create_pw(trial, layer_id, max_stride):
    num_filters = trial.suggest_int(f"layer_{layer_id}_pwconv2d_filters", 4, 32, step=4)
    stride_size = trial.suggest_int(f"layer_{layer_id}_pwconv2d_ssize", 2, max_stride)
    return layers.Conv2D(num_filters, 1, stride_size)


def create_layer(trial, layer_type, layer_id, input_size):
    max_stride = 4

    if layer_type == "Conv2D":
        layer = create_conv2d(trial, layer_id, max_stride, input_size)

    elif layer_type == "Depthwise2D":
        layer = create_dw(trial, layer_id, max_stride, input_size)

    elif layer_type == "Pointwise2D":
        layer = create_pw(trial, layer_id, max_stride)

    return layer


def create_augmentation_layers(trial, x):
    # augmix_severity = trial.suggest_float("augmix_severity", 0.0, 0.5)
    brightness_factor = trial.suggest_float("brightness_factor", 0, 0.2)
    gaussian_noise_factor = trial.suggest_float("gaussian_noise_factor", 0, 0.2)
    translate_factor = trial.suggest_float("translate_factor", 0, 0.2)

    # x = keras_cv.layers.AugMix(value_range=(0, 1), severity=augmix_severity)(x)
    x = layers.RandomBrightness(brightness_factor, (0, 1.0))(x)
    x = layers.RandomFlip()(x)
    x = layers.RandomTranslation(translate_factor, translate_factor)(x)
    x = layers.GaussianNoise(gaussian_noise_factor)(x)

    return x


def create_dense_output(trial, x):
    do_factor = trial.suggest_float(
        "output_dropout", dropout_range[0], dropout_range[1]
    )

    # output layer
    x = layers.Flatten()(x)
    x = layers.Dropout(do_factor)(x)
    x = layers.Dense(num_classes)(x)
    return x



def create_hidden_layers(trial, x):
    num_layers = trial.suggest_categorical("num_layers", num_layer_options)

    # create num_layers...
    for i in range(num_layers - 1):

        layer_type = trial.suggest_categorical(f"layer_{i}", layer_options)
        act_selected = trial.suggest_categorical(f"activation_{i}", activation_options)

        layer = create_layer(
            trial,
            layer_type,
            i,
            input_size=x.shape[1],
        )
        x = layer(x)
        x = layers.Activation(act_selected)(x)

    return x


def calibrate_conv_parameters(input_size, s, desired_fmap_size):    
    b, h, w, c = input_size

    h = h // s
    w = w // s

    # Reduzir o número de canais se necessário
    while np.prod([h, w, c]) > desired_fmap_size:
        c -= 1

    # Aumentar o número de canais até atender ao tamanho máximo desejado
    while np.prod([h, w, c]) < desired_fmap_size:
        c += 1

    c -= 1  # Reduzir um canal para garantir que o tamanho seja no máximo o desejado

    if c <= 0:
        c = 1
        input_size = [None, h, w, c]
        h, w, c, s = calibrate_conv_parameters(input_size, s + 1, desired_fmap_size)

    return h, w, c, s


def create_dynamic_layer(trial, x):
    fmap_max_size = trial.suggest_int(
        "fmap_max_size", fmap_size_range[0], fmap_size_range[1], step=fmap_size_range[2]
    )
    print("fmap_max_size", fmap_max_size)
    dynamic_kernel_size = trial.suggest_categorical(
        f"dynamic_layer_conv2d_fsize", filter_size_options
    )

    h, w, filters, stride = calibrate_conv_parameters(x.shape, 1, fmap_max_size)

    print("h, w, filters, stride", h, w, filters, stride)

    dynamic_kernel_size = clip_filter_size(dynamic_kernel_size, h)

    # dynamic layer
    x = layers.Conv2D(filters, dynamic_kernel_size, strides=stride)(x)

    return x


def create_model(trial, input_size):
    x = x_in = layers.Input(shape=(input_size, input_size, 3))
    x = create_augmentation_layers(trial, x)
    x = create_hidden_layers(trial, x)
    x = create_dynamic_layer(trial, x)
    x = create_dense_output(trial, x)

    return keras.Model(x_in, x)

# if testing


```
h, w, c, s = calibrate_conv_parameters([None, 63, 63, 12], 1, 640)
print(h, w, c, s)
print(np.prod([h, w, c]))


class MockTrial:
    def suggest_int(self, name, min, max, step=None, log=False):
        if step is None:
            return np.random.randint(min, max)
        range_ = np.arange(min, max, step, np.int32)
        i = range_[np.random.randint(0, len(range_))]
        print("iii", i)
        return i

    def suggest_float(self, name, min, max, step=None, log=False):
        return np.random.uniform(min, max)

    def suggest_categorical(self, name, options):
        return options[np.random.randint(0, len(options))]

    def suggest_discrete_uniform(self, name, min, max, q):
        return (np.random.randint(min, max) // q) * q


test_input_size = 128
test_batch_size = 64

# load data
train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, test_batch_size, test_input_size
)

model = create_model(MockTrial(), test_input_size)
model.build(input_shape=(test_batch_size, test_input_size, test_input_size, 3))
model.summary()
model.save("saved_models/test/")
model.compile(
    optimizer="adam",
    loss=keras.losses.CategoricalCrossentropy(True),
    metrics=["accuracy"],
)
model.evaluate(train_dataset)
model.evaluate(test_dataset)



```

In [6]:
def create_optimizer(trial):
    lr = trial.suggest_float("lr_inicial", 3e-4, 2e-3, log=True)
    optimizer = keras.optimizers.Adam(lr)
    return optimizer


def create_model_checkpointer(save_path):
    return keras.callbacks.ModelCheckpoint(
        save_path + "weights/",
        "val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )


def crate_early_stopper():
    return keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0.1,
        patience=20,
        verbose=0,
        mode="max",
        start_from_epoch=50,
    )


def count_parameters(model):
    total_params = 0
    for layer in model.trainable_variables:
        layer_params = tf.reduce_prod(layer.shape)
        total_params += layer_params
    return total_params.numpy()


def objective(trial):
    batch_size = trial.suggest_int("batchsize", 32, 128, step=32)
    input_size = trial.suggest_int("inputsize", 64, 128, step=16)

    # load data
    train_dataset, test_dataset, num_classes = dataset.create_datasets(
        train_images_rgx, test_images_rgx, batch_size, input_size
    )

    # Build model and optimizer.
    model = create_model(trial, input_size)
    assert isinstance(model, keras.Model), "model is not a model"

    optimizer = create_optimizer(trial)

    # compile the model
    model.compile(
        optimizer, loss=keras.losses.categorical_crossentropy, metrics=["accuracy"]
    )

    # crate model checkpointer
    saved_best_model_path = f"saved_models/{EXP_ID}/best_model_t{trial._trial_id}/"
    model.save(saved_best_model_path, True, "tf")  # save all model

    model_chkp = create_model_checkpointer(saved_best_model_path)
    early_stopper = crate_early_stopper()

    # Training and validating cycle.
    history = model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=EPOCHS,
        callbacks=[model_chkp, early_stopper],
        verbose="auto",
    )

    # load best model
    model.load_weights(saved_best_model_path + "weights/")

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(test_dataset, verbose=0)

    # get the model size
    model_size = count_parameters(model)

    # Return best model validation accuracy.
    return score[1]


def search():
    study = optuna.create_study(
        direction="maximize",
        sampler=optuna.samplers.TPESampler(),
    )
    study.optimize(objective, n_trials=NTRIALS)
    return study

# start search

In [7]:
EXP_ID = "exp8_no_depth_w_GAP_multiObj"
keras.utils.disable_interactive_logging()

# start searching for a model
study = search()

# save the study
now = datetime.now()
joblib.dump(study, "saved_studies/" + now.strftime("%Y:%m:%d:%H:%M:%S") + ".pkl")

[I 2024-02-04 21:09:56,928] A new study created in memory with name: no-name-49c01565-e81c-4a10-ab50-2d60bf9947bf


fmap_max_size 576
h, w, filters, stride 19 19 1 2
INFO:tensorflow:Assets written to: saved_models/exp8_no_depth_w_GAP_multiObj/best_model_t0/assets


INFO:tensorflow:Assets written to: saved_models/exp8_no_depth_w_GAP_multiObj/best_model_t0/assets
I0000 00:00:1707091800.364555  477044 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
[I 2024-02-04 21:10:07,126] Trial 0 finished with value: 0.26923078298568726 and parameters: {'batchsize': 64, 'inputsize': 80, 'brightness_factor': 0.09446518909959155, 'gaussian_noise_factor': 0.12505344981627553, 'translate_factor': 0.048785535050076816, 'num_layers': 2, 'layer_0': 'Depthwise2D', 'activation_0': 'linear', 'layer_0_dwconv2d_fsize': 4, 'layer_0_dwconv2d_ssize': 2, 'fmap_max_size': 576, 'dynamic_layer_conv2d_fsize': 3, 'output_dropout': 0.5051643311368255, 'lr_inicial': 0.0013860885992572599}. Best is trial 0 with value: 0.26923078298568726.


fmap_max_size 576
h, w, filters, stride 18 18 1 2
INFO:tensorflow:Assets written to: saved_models/exp8_no_depth_w_GAP_multiObj/best_model_t1/assets


INFO:tensorflow:Assets written to: saved_models/exp8_no_depth_w_GAP_multiObj/best_model_t1/assets
[I 2024-02-04 21:10:16,308] Trial 1 finished with value: 0.2846153974533081 and parameters: {'batchsize': 96, 'inputsize': 112, 'brightness_factor': 0.1499588100072999, 'gaussian_noise_factor': 0.11875365736815346, 'translate_factor': 0.01755169048771261, 'num_layers': 2, 'layer_0': 'Depthwise2D', 'activation_0': 'linear', 'layer_0_dwconv2d_fsize': 5, 'layer_0_dwconv2d_ssize': 3, 'fmap_max_size': 576, 'dynamic_layer_conv2d_fsize': 5, 'output_dropout': 0.331486900981135, 'lr_inicial': 0.0009354212195521014}. Best is trial 1 with value: 0.2846153974533081.


...

fmap_max_size 896
h, w, filters, stride 19 19 2 2
INFO:tensorflow:Assets written to: saved_models/exp8_no_depth_w_GAP_multiObj/best_model_t798/assets


INFO:tensorflow:Assets written to: saved_models/exp8_no_depth_w_GAP_multiObj/best_model_t798/assets
[I 2024-02-04 23:06:53,471] Trial 798 finished with value: 0.1846153885126114 and parameters: {'batchsize': 96, 'inputsize': 80, 'brightness_factor': 0.004326574747186194, 'gaussian_noise_factor': 0.1618789509225441, 'translate_factor': 0.08541799449945754, 'num_layers': 2, 'layer_0': 'Conv2D', 'activation_0': 'linear', 'layer_0_conv2d_filters': 12, 'layer_0_conv2d_fsize': 3, 'layer_0_conv2d_ssize': 2, 'fmap_max_size': 896, 'dynamic_layer_conv2d_fsize': 4, 'output_dropout': 0.5431153416615854, 'lr_inicial': 0.0004522232868853281}. Best is trial 642 with value: 0.6230769157409668.


fmap_max_size 896
h, w, filters, stride 5 5 35 1
INFO:tensorflow:Assets written to: saved_models/exp8_no_depth_w_GAP_multiObj/best_model_t799/assets


INFO:tensorflow:Assets written to: saved_models/exp8_no_depth_w_GAP_multiObj/best_model_t799/assets
[I 2024-02-04 23:07:01,093] Trial 799 finished with value: 0.29230770468711853 and parameters: {'batchsize': 96, 'inputsize': 80, 'brightness_factor': 0.014334758426332951, 'gaussian_noise_factor': 0.18290174670713988, 'translate_factor': 0.06733265382928687, 'num_layers': 3, 'layer_0': 'Pointwise2D', 'activation_0': 'relu', 'layer_0_pwconv2d_filters': 16, 'layer_0_pwconv2d_ssize': 4, 'layer_1': 'Pointwise2D', 'activation_1': 'linear', 'layer_1_pwconv2d_filters': 8, 'layer_1_pwconv2d_ssize': 4, 'fmap_max_size': 896, 'dynamic_layer_conv2d_fsize': 4, 'output_dropout': 0.4800755035048645, 'lr_inicial': 0.00035277202094566675}. Best is trial 642 with value: 0.6230769157409668.


FileNotFoundError: [Errno 2] No such file or directory: 'saved_studies/2024:02:04:23:07:01.pkl'

In [8]:
# save the study
now = datetime.now()
joblib.dump(study, "saved_studies/" + now.strftime("%Y%m%d%H%M%S") + ".pkl")

['saved_studies/20240205131809.pkl']

# visualizations

In [9]:
print("Number of finished trials: ", len(study.trials))

trials = study.best_trials
for trial in trials:
    print(f"Best trial id: {trial._trial_id}")
    
    print("  Value: ", trial.values)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

Number of finished trials:  800
Best trial id: 642
  Value:  [0.6230769157409668]
  Params: 
    batchsize: 96
    inputsize: 80
    brightness_factor: 0.007593516696413408
    gaussian_noise_factor: 0.08860534383651557
    translate_factor: 0.07792744949556095
    num_layers: 3
    layer_0: Pointwise2D
    activation_0: relu
    layer_0_pwconv2d_filters: 16
    layer_0_pwconv2d_ssize: 4
    layer_1: Pointwise2D
    activation_1: linear
    layer_1_pwconv2d_filters: 8
    layer_1_pwconv2d_ssize: 4
    fmap_max_size: 896
    dynamic_layer_conv2d_fsize: 4
    output_dropout: 0.513942187489966
    lr_inicial: 0.00047672998249735627


In [ ]:
optuna.visualization.plot_pareto_front(study, target_names=["accuracy", "model_size"])

In [ ]:
study_name = "2024:02:02:14:37:58.pkl"
study_2 = joblib.load(
    f"/home/webphy/Desktop/dnn_processor/neural_network/automl/saved_studies/{study_name}"
)

In [11]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [12]:
plot_optimization_history(study)

In [13]:
plot_param_importances(study)

In [14]:
plot_parallel_coordinate(
    study,
    [
        "lr_inicial",
        "num_layers",
        "layer_0",
        "activation_0",
    ],
)